In [1]:
from bs4 import BeautifulSoup
from bs4.element import Comment
import re
import pandas as pd
import numpy as np
import nltk
import html5lib
import glob
import codecs
import os
import cltk
from cltk.tokenize.word import  WordTokenizer

In [2]:
print ("The current working directory is", os.getcwd())

The current working directory is /Users/mateomontoya/Desktop/work/projects_ongoing/flaubert2


In [3]:
flaubert_data = pd.read_csv('parsed.csv')
len(flaubert_data)

FileNotFoundError: [Errno 2] File b'parsed.csv' does not exist: b'parsed.csv'

In [ ]:
flaubert_data.head()

In [ ]:
word_tokenizer = WordTokenizer('french')

In [ ]:

def token_flaubert_data(column, dataframe):
    folios_tokenized = []

    for text in dataframe[column]:
        if type(text)==str:
            tokens = word_tokenizer.tokenize(text)
            folios_tokenized.append(tokens)
        else:
            folios_tokenized.append(' ')
    print(folios_tokenized[0])
    return folios_tokenized

In [ ]:
flaubert_text = token_flaubert_data('text', flaubert_data)

In [ ]:
type(flaubert_data['text'][0])

In [ ]:
flaubert_data['chapter'].value_counts(dropna=False)

In [ ]:
len(flaubert_data['chapter'])

In [ ]:
print(flaubert_data['text'][0])

In [ ]:
flaubert_data['text_tokenized'] = flaubert_text

In [ ]:
flaubert_text = flaubert_data[['chapter', 'version','folio', 'text', 'text_tokenized']]

In [ ]:
flaubert_text.head()

In [ ]:
flaubert_chap_1 = flaubert_text[flaubert_text['chapter'] == 1]

In [ ]:
flaubert_chap_1.head(20)

In [ ]:
flaubert_chap_1.describe()

In [ ]:
flaubert_chap_1['version'].unique()

In [ ]:
flaubert_versions = flaubert_data[['chapter', 'version', 'text_tokenized']]

In [ ]:
flaubert_versions['version'].value_counts(dropna=False)

In [ ]:
flaubert_versions[flaubert_versions['version'] == 'C'].head()

In [ ]:
def combine_tokens_on_version(data, version_tag, extended_column):
    versions = ['1', '2', '3', '4', '5', '6', 'C', 'D']
    version_text = []
    index = 0
    for version in data['version']:
        if version_tag == version:
            version_text.extend(data[extended_column][index])
        index += 1
    return version_text
            
            

In [ ]:
version_1 = combine_tokens_on_version(flaubert_versions, '1', 'text_tokenized')
version_2 = combine_tokens_on_version(flaubert_versions, '2', 'text_tokenized')
version_3 = combine_tokens_on_version(flaubert_versions, '3', 'text_tokenized')
version_4 = combine_tokens_on_version(flaubert_versions, '4', 'text_tokenized')
version_5 = combine_tokens_on_version(flaubert_versions, '5', 'text_tokenized')
version_6 = combine_tokens_on_version(flaubert_versions, '6', 'text_tokenized')
version_c = combine_tokens_on_version(flaubert_versions, 'C', 'text_tokenized')
version_d = combine_tokens_on_version(flaubert_versions, 'D', 'text_tokenized')
print(version_1[:5])
print(version_c[:5])

In [ ]:
data = {'version': ['1', '2', '3', '4', '5', '6', 'C', 'D'],
        'token_text': [version_1, version_2, version_3, version_4, version_5, version_6, version_c, version_d]}

In [ ]:
flaubert_version = pd.DataFrame(data=data)
flaubert_version

In [ ]:
flaubert_version['token_text']

In [ ]:
flaubert_version[flaubert_version['version'] == '1']['token_text']

In [ ]:
draft_1 = pd.DataFrame([w for w in flaubert_version[flaubert_version['version'] == '1']['token_text']])
draft_1 = draft_1.T
draft_1['Tokens'] = draft_1[0]
draft_1 = draft_1.drop(columns=[0])
draft_1.head()

In [ ]:
draft_1['Tokens'].value_counts()

show the length of edits and of erasures -- find a way to find the most edited words

In [ ]:
flaubert_data.head()

In [ ]:
struck_through_tokens = token_flaubert_data('struck', flaubert_data)
margins_tokens = token_flaubert_data('margins', flaubert_data)
margins_no_struck_tokens = token_flaubert_data('margins_no_struck', flaubert_data)
margins_struck_tokens = token_flaubert_data('margins_struck', flaubert_data)
previous_struck_tokens = token_flaubert_data('previous_struck', flaubert_data)

In [ ]:
flaubert_data['struck_tokens'] = struck_through_tokens
flaubert_data['margins_tokens'] = margins_tokens
flaubert_data['margins_no_struck_tokens'] = margins_no_struck_tokens
flaubert_data['margins_struck_tokens'] = margins_struck_tokens
flaubert_data['previous_struck_tokens'] = previous_struck_tokens

In [ ]:
flaubert_data.head()

In [ ]:
flaubert_struck_through = flaubert_data[['version', 'chapter', 'folio',
                                        'struck', 'struck_tokens', 
                                        'previous_struck', 'previous_struck_tokens',
                                        'margins', 'margins_tokens',
                                        'margins_no_struck', 'margins_no_struck_tokens',
                                        'margins_struck', 'margins_struck_tokens']]

In [ ]:
extend_columns_list = ['struck_tokens', 'previous_struck_tokens',
                       'margins_tokens', 'margins_no_struck_tokens',
                      'margins_struck_tokens']

In [ ]:
def combine_5_tokens_on_version(data, version_tag, extended_column_list):
    versions = ['1', '2', '3', '4', '5', '6', 'C', 'D']
    version_text = []
    index = 0
    for version in data['version']:
        if version_tag == version:
            version_text.extend(data[extended_column_list[0]][index])
            version_text.extend(data[extended_column_list[1]][index])
            version_text.extend(data[extended_column_list[2]][index])
            version_text.extend(data[extended_column_list[3]][index])
            version_text.extend(data[extended_column_list[4]][index])
        index += 1
    return version_text

## Struck Through Analysis: Comparing Different Versions

By looking at every word that Flaubert and his copyiste struck through in Madame Bovary, we can see the differences in both the frequency and types of words that were edited.

In [ ]:
version_1 = combine_tokens_on_version(flaubert_struck_through, '1', 'struck_tokens')
version_2 = combine_tokens_on_version(flaubert_struck_through, '2', 'struck_tokens')
version_3 = combine_tokens_on_version(flaubert_struck_through, '3', 'struck_tokens')
version_4 = combine_tokens_on_version(flaubert_struck_through, '4', 'struck_tokens')
version_5 = combine_tokens_on_version(flaubert_struck_through, '5', 'struck_tokens')
version_6 = combine_tokens_on_version(flaubert_struck_through, '6', 'struck_tokens')
version_c = combine_tokens_on_version(flaubert_struck_through, 'C', 'struck_tokens')
version_d = combine_tokens_on_version(flaubert_struck_through, 'D', 'struck_tokens')

In [ ]:
version_1[:25]

In [ ]:
data = {'version': ['1', '2', '3', '4', '5', '6', 'C', 'D'],
        'struck_tokens': [version_1, version_2, version_3, version_4, version_5, version_6, version_c, version_d]}





In [ ]:
flaubert_struck_versions = pd.DataFrame(data)

In [ ]:
flaubert_struck_versions.head(25)

In [ ]:
version_1_strikes_len = len(flaubert_struck_versions['struck_tokens'][0])
version_1_strikes_len

In [ ]:
version_2_strikes_len = len(flaubert_struck_versions['struck_tokens'][1])
version_2_strikes_len

In [ ]:
version_3_strikes_len = len(flaubert_struck_versions['struck_tokens'][2])
version_3_strikes_len

In [ ]:
version_4_strikes_len = len(flaubert_struck_versions['struck_tokens'][3])
version_4_strikes_len

In [ ]:
version_5_strikes_len = len(flaubert_struck_versions['struck_tokens'][4])
version_5_strikes_len

In [ ]:
version_6_strikes_len = len(flaubert_struck_versions['struck_tokens'][5])
version_6_strikes_len

In [ ]:
version_C_strikes_len = len(flaubert_struck_versions['struck_tokens'][6])
version_C_strikes_len

In [ ]:
version_D_strikes_len = len(flaubert_struck_versions['struck_tokens'][7])
version_D_strikes_len

In [ ]:
strike_lengths = [version_1_strikes_len, version_2_strikes_len, version_3_strikes_len,
                  version_4_strikes_len, version_5_strikes_len, version_6_strikes_len,
                 version_C_strikes_len, version_D_strikes_len]

versions = ['Brouillon 1', 'Brouillon 2', 'Brouillon 3', 'Brouillon 4', 'Brouillon 5', 'Brouillon 6', 'Copiste', 'Definitif']


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
## number of words struck through in each version
plt.figure(figsize=[14, 10])
plt.xlabel('Version')
plt.ylabel('Number of Struck Through Words')
plt.title('Frequency of Struck Through Words in Each Version of Madame Bovary')
sns.barplot(x=versions, y=strike_lengths);


## Unedited Previous Text (prev_no_struck) Exploration: Comparing Different Versions

In [ ]:
def initialize_version_table(column, original_frame):
    version_key_list = ['1', '2', '3', '4', '5', '6', 'C', 'D']
    
    
    def token_flaubert_data(column, dataframe):
        folios_tokenized = []

        for text in dataframe[column]:
            if type(text)==str:
                tokens = word_tokenizer.tokenize(text)
                folios_tokenized.append(tokens)
            else:
                folios_tokenized.append(' ')
        return folios_tokenized

    tokenized_txt = token_flaubert_data(column, original_frame)
    original_frame['tokens'] = tokenized_txt
    original_frame = original_frame[['version', 'tokens']]
    
    version_1 = original_frame[original_frame['version'] == '1']
    version_2 = original_frame[original_frame['version'] == '2']
    version_3 = original_frame[original_frame['version'] == '3']
    version_4 = original_frame[original_frame['version'] == '4']
    version_5 = original_frame[original_frame['version'] == '5']
    version_6 = original_frame[original_frame['version'] == '6']
    version_C = original_frame[original_frame['version'] == 'C']
    version_D = original_frame[original_frame['version'] == 'D']
    
    versions = [version_1, version_2, version_3, version_4, version_5, version_6, version_C, version_D]
    
    def concat_text(version_table):
        full_text = []
        for row in version_table['tokens']:
            full_text += row
        
        return full_text
    
    concat_versions = [concat_text(version) for version in versions]
    
    final_frame = pd.DataFrame({'Version': version_key_list, 'Tokens': concat_versions})
    
    return final_frame
    
    
    
#     def combine_tokens_on_version(data, version_tag, extended_column):
#         version_text = []
#         index = 0
#         for version in data['version']:
#             if version_tag == version:
#                 version_text.extend(data[extended_column][index])
#             index += 1
#         return version_text
            
    
#     def collapse_versions(data, extended_column):
#         versions = ['1', '2', '3', '4', '5', '6', 'C', 'D']
#         full_version_tokens = []
#         for version in versions:
#             version_texty = combine_tokens_on_version(original_frame, version, 'tokens')
#             full_version_tokens += version_texty
                
#         #version_frame = pd.DataFrame({'Version': version_key_list, 'Tokens': full_version_tokens}) 
#         return full_version_tokens
    
#     collapse_versions(original_frame, 'tokens')
    
    

In [ ]:
prev_no_struck_tokens = initialize_version_table('previous_no_struck', flaubert_data)
prev_no_struck_tokens

In [ ]:
def len_punct_versions(data, text_column):
    list_punct = list(string.punctuation)
    len_punct_list = []
    for version in np.arange(8):
        version_tab = data[text_column][version]
        punct = [token for token in version_tab if token in list_punct]
        len_punct = len(punct)
        len_punct_list.append(len_punct)
    return len_punct_list
    
    

In [ ]:
len_punct_previous_no_struck = len_punct_versions(prev_no_struck_tokens, 'Tokens')

In [ ]:
plt.figure(figsize=[14, 10])
plt.title('Length of Total Unedited Punctuation')
sns.barplot(x=versions, y=len_punct_previous_no_struck);

## Margin Exploration: Comparing Different Versions

By looking at the margins of the text, where words and comments were added to the text, we can where Flaubert and his copyiste focused their editing efforts.

In [ ]:
version_1 = combine_tokens_on_version(flaubert_struck_through, '1', 'margins_tokens')
version_2 = combine_tokens_on_version(flaubert_struck_through, '2', 'margins_tokens')
version_3 = combine_tokens_on_version(flaubert_struck_through, '3', 'margins_tokens')
version_4 = combine_tokens_on_version(flaubert_struck_through, '4', 'margins_tokens')
version_5 = combine_tokens_on_version(flaubert_struck_through, '5', 'margins_tokens')
version_6 = combine_tokens_on_version(flaubert_struck_through, '6', 'margins_tokens')
version_c = combine_tokens_on_version(flaubert_struck_through, 'C', 'margins_tokens')
version_d = combine_tokens_on_version(flaubert_struck_through, 'D', 'margins_tokens')

In [ ]:
data = {'version': ['1', '2', '3', '4', '5', '6', 'C', 'D'],
        'margins_tokens': [version_1, version_2, version_3, version_4, version_5, version_6, version_c, version_d]}

In [ ]:
flaubert_margins_versions = pd.DataFrame(data)

In [ ]:
flaubert_margins_versions

In [ ]:

version_c_margins_len = len(flaubert_margins_versions['margins_tokens'][6])
print(version_c_margins_len)


In [ ]:
testy = [token for token in flaubert_margins_versions['margins_tokens'][1] if token in (list(string.punctuation))]

In [ ]:
def len_punct_margins(key):
    list_punct = list(string.punctuation)
    version_tab = flaubert_margins_versions['margins_tokens'][key]
    punct = [token for token in version_tab if token in list_punct]
    return len(punct)

In [ ]:
version_key_list = ['1', '2', '3', '4', '5', '6', 'C', 'D']
num_keys = np.arange(8)

In [ ]:
len_punct_margin = [len_punct_margins(version) for version in num_keys]
len_punct_margin

In [ ]:
plt.figure(figsize=[14, 10])
plt.title('Length of Total Punctuation in Margin')
sns.barplot(x=versions, y=len_punct_margin);

In [ ]:
def gen_len(data, column):
    len_list = []
    for tokens in data[column]:
        len_list.append(len(tokens))
    return len_list
            
            

In [ ]:
len_margins_list = gen_len(flaubert_margins_versions, 'margins_tokens')
len_margins_list

In [ ]:
## number of words in margins in each version
plt.figure(figsize=[14, 10])
plt.xlabel('Version')
plt.ylabel('Total Number of Words in Margin')
plt.title('Frequency of Words in Margins of Each Version of Madame Bovary')
sns.barplot(x=versions, y=len_margins_list);

In [ ]:
strike_lengths + len_margins_list

In [ ]:
struck_df = pd.DataFrame({'Version': versions,
                                 'Frequency': strike_lengths,
                                 'Hue': ['Total Struck Through Text' for _ in np.arange(8)]})

In [ ]:
margin_df = pd.DataFrame({'Version': versions,
                                 'Frequency': len_margins_list,
                                 'Hue': ['Total Margin Text' for _ in np.arange(8)]})

In [ ]:
margin_struck_df = struck_df.append(margin_df)
margin_struck_df

In [ ]:
plt.figure(figsize=[14, 10])
plt.xlabel('Version')
plt.ylabel('Number of Words')
plt.title('Comparison of Frequency of Words Crossed Out v. in Margins of Each Version of Madame Bovary')
sns.barplot(data= margin_struck_df, x='Version', y='Frequency', hue='Hue');

In [ ]:
margins_no_struck_df = initialize_version_table('margins_no_struck', flaubert_data)

In [ ]:
def len_versions(data, text_column):
    len_list = []
    for version in np.arange(8):
        len_version = len(data[text_column][version])
        len_list.append(len_version)
    return len_list

In [ ]:
len_margins_no_struck = len_versions(margins_no_struck_df, 'Tokens')

In [ ]:
margin_unedited_df = pd.DataFrame({'Version': versions,
                                 'Frequency': len_margins_no_struck,
                                 'Hue': ['Unedited Margin Text' for _ in np.arange(8)]})

In [ ]:
margin_struck_unedit_df = struck_df.append(margin_unedited_df)

In [ ]:
plt.figure(figsize=[14, 10])
plt.xlabel('Version')
plt.ylabel('Number of Words')
plt.title('Comparison of Frequency of Words Crossed Out v. in Margins of Each Version of Madame Bovary')
sns.barplot(data= margin_struck_unedit_df, x='Version', y='Frequency', hue='Hue');

In [ ]:
draft_1 = pd.DataFrame([w for w in flaubert_version[flaubert_version['version'] == '1']['token_text']])
draft_1 = draft_1.T
draft_1['Tokens'] = draft_1[0]
draft_1 = draft_1.drop(columns=[0])
draft_1.head()

In [ ]:
brouillons_1 = pd.DataFrame([w for w in flaubert_struck_versions[flaubert_struck_versions['version'] == '1']['struck_tokens']])
brouillons_1 = brouillons_1.T
brouillons_1['struck_tokens'] = brouillons_1[0]
brouillons_1 = brouillons_1.drop(columns=[0])
brouillons_1.head()

In [ ]:
brouillons_2 = pd.DataFrame([w for w in flaubert_struck_versions[flaubert_struck_versions['version'] == '2']['struck_tokens']])
brouillons_2 = brouillons_2.T
brouillons_2['struck_tokens'] = brouillons_2[0]
brouillons_2 = brouillons_2.drop(columns=[0])
brouillons_2.head()


In [ ]:
brouillons_3 = pd.DataFrame([w for w in flaubert_struck_versions[flaubert_struck_versions['version'] == '3']['struck_tokens']])
brouillons_3 = brouillons_3.T
brouillons_3['struck_tokens'] = brouillons_3[0]
brouillons_3 = brouillons_3.drop(columns=[0])
brouillons_3.head()


In [ ]:
brouillons_4 = pd.DataFrame([w for w in flaubert_struck_versions[flaubert_struck_versions['version'] == '4']['struck_tokens']])
brouillons_4 = brouillons_4.T
brouillons_4['struck_tokens'] = brouillons_4[0]
brouillons_4 = brouillons_4.drop(columns=[0])
brouillons_4.head()


In [ ]:
brouillons_5 = pd.DataFrame([w for w in flaubert_struck_versions[flaubert_struck_versions['version'] == '5']['struck_tokens']])
brouillons_5 = brouillons_5.T
brouillons_5['struck_tokens'] = brouillons_5[0]
brouillons_5 = brouillons_5.drop(columns=[0])
brouillons_5.head()

In [ ]:
brouillons_c = pd.DataFrame([w for w in flaubert_struck_versions[flaubert_struck_versions['version'] == 'C']['struck_tokens']])
brouillons_c = brouillons_c.T
brouillons_c['struck_tokens'] = brouillons_c[0]
brouillons_c = brouillons_c.drop(columns=[0])
brouillons_c.head()


In [ ]:
brouillons_d = pd.DataFrame([w for w in flaubert_struck_versions[flaubert_struck_versions['version'] == 'D']['struck_tokens']])
brouillons_d = brouillons_d.T
brouillons_d['struck_tokens'] = brouillons_d[0]
brouillons_d = brouillons_d.drop(columns=[0])
brouillons_d.head()

In [ ]:
print ("The current working directory is", os.getcwd())

In [ ]:
from nltk.tag import StanfordPOSTagger as POS_Tag

home = '/Users/mateomontoya/Desktop'

_path_to_model = home + '/stanford-postagger/models/french.tagger' 
_path_to_jar = home + '/stanford-postagger/stanford-postagger.jar'
st = POS_Tag(_path_to_model, _path_to_jar)

In [ ]:
st.tag("Je n'aime pas Madame Bovary de Flaubert".split())

In [ ]:
pos_tag_draft_1 = st.tag(brouillons_1['struck_tokens'])

In [ ]:
pos_tag_draft_1[:20]

In [ ]:
pos_tag_brouillons_1 = pd.DataFrame({'Tokens': [w[0] for w in pos_tag_draft_1],
                                     'POS_Tag': [w[1] for w in pos_tag_draft_1]})

In [ ]:
pos_tag_brouillons_1.head()

In [ ]:
pos_tag_brouillons_1['POS_Tag'].value_counts()

In [ ]:
b_1_pos_count = pd.DataFrame(pos_tag_brouillons_1['POS_Tag'].value_counts()[:15])
b_1_pos_count.reset_index(inplace=True)
b_1_pos_count.head(10)

In [ ]:
selected_POS_tags = ['P', 'NC', 'DET', 'V', 'ADV', 'PUNC', 'CLS', 'N', 'CC', 'ADJ', 'VINF', 'VPP', 'CLO', 'PRO', 'CLR']

In [ ]:
b_1_pos_count_select = b_1_pos_count[b_1_pos_count['index'].isin(selected_POS_tags)]
b_1_pos_count_select = b_1_pos_count_select.rename(index=str, columns={'POS_Tag': 'Count', 'index': 'POS_Tag'})
b_1 = ['Brouillons_1' for _ in range(14)]
b_1_pos_count_select['Version'] = b_1
b_1_pos_count_select.head()

In [ ]:
pos_tag_definitif = st.tag(brouillons_d['struck_tokens'])

In [ ]:
pos_tag_definitif_data = pd.DataFrame({'Tokens': [w[0] for w in pos_tag_definitif],
                                     'POS_Tag': [w[1] for w in pos_tag_definitif]})

In [ ]:
pos_tag_definitif_data.head()

In [ ]:
pos_tag_definitif_data['POS_Tag'].value_counts()

In [ ]:
def_pos_count = pd.DataFrame(data=pos_tag_definitif_data['POS_Tag'].value_counts())
def_pos_count = def_pos_count.reset_index()
def_pos_count.head(20)


In [ ]:
def_pos_count_select = def_pos_count[def_pos_count['index'].isin(selected_POS_tags)]
def_pos_count_select = def_pos_count_select.rename(index=str, columns={'POS_Tag': 'Count', 'index': 'POS_Tag'})
definitif = ['Definitif' for _ in range(15)]
def_pos_count_select['Version'] = definitif
def_pos_count_select.head()

In [ ]:
POS_counts_appended = b_1_pos_count_select.append(def_pos_count_select, ignore_index=True)
POS_counts_appended

In [ ]:
POS_counts_merged = b_1_pos_count_select.merge(def_pos_count_select, on='POS_Tag')
POS_counts_merged

In [ ]:
plt.figure(figsize=[14, 10])
plt.title('Compared POS Tag Frequency of Top 15 POS Tags of Struck Words in First Draft and Final Edition')
sns.barplot(data=POS_counts_appended, x='POS_Tag', y='Count', hue='Version');

In [ ]:
plt.figure(figsize=[14, 10])
plt.title('Compared POS Tag Frequency of Top 15 POS Tags in Final Draft')
sns.barplot(x=POS_counts_merged['POS_Tag'], y=POS_counts_merged['Count_y']);

Nouns, Adjectives, Proper Nouns, Adverbs, Punctuation, Conjunctions - The Frugal Editor -- Naddaff

We distinguish 15 lexical categories, used for simple words as well as for compounds:

- A (adjective)
- Adv (adverb)
- CC (coordinating conjunction)
- Cl (weak clitic pronoun)
- CS (subordinating conjunction)
- D (determiner)
- ET (foreign word)
- I (interjection)
- NC (common noun)
- NP (proper noun)
- P (preposition)
- PREF (prefix)
- PRO (strong pronoun)
- V (verb)
- PONCT (punctuation mark)

Our phrasal tagset is as follows:

- AP (adjectival phrases)
- AdP (adverbial phrases)
- COORD (coordinated phrases)
- NP (noun phrases)
- PP (prepositional phrases)
- VN (verbal nucleus)
- VPinf (infinitive clauses)
- VPpart (nonfinite clauses)
- SENT (sentences)
- Sint, Srel, Ssub (finite clauses)

## Full Text (Margin and Struckthrough Analysis)

In [ ]:
draft_1 = pd.DataFrame([w for w in flaubert_version[flaubert_version['version'] == '1']['token_text']])
draft_1 = draft_1.T
draft_1['Tokens'] = draft_1[0]
draft_1 = draft_1.drop(columns=[0])
draft_1.head()

In [ ]:
draft_1 = pd.DataFrame([w for w in flaubert_version[flaubert_version['version'] == '1']['token_text']])
draft_1 = draft_1.T
draft_1['Tokens'] = draft_1[0]
draft_1 = draft_1.drop(columns=[0])
draft_1.head()

In [ ]:
import string

In [ ]:
listy = list(string.punctuation)

In [ ]:
draft_1_punct = draft_1[draft_1['Tokens'].isin(listy)]
len(draft_1_punct)

In [ ]:
draft_2 = pd.DataFrame([w for w in flaubert_version[flaubert_version['version'] == '2']['token_text']])
draft_2 = draft_2.T
draft_2['Tokens'] = draft_2[0]
draft_2 = draft_2.drop(columns=[0])
draft_2.head()

In [ ]:
draft_3 = pd.DataFrame([w for w in flaubert_version[flaubert_version['version'] == '3']['token_text']])
draft_3 = draft_3.T
draft_3['Tokens'] = draft_3[0]
draft_3 = draft_3.drop(columns=[0])
draft_3.head()

In [ ]:
draft_4 = pd.DataFrame([w for w in flaubert_version[flaubert_version['version'] == '4']['token_text']])
draft_4 = draft_4.T
draft_4['Tokens'] = draft_4[0]
draft_4 = draft_4.drop(columns=[0])
draft_4.head()

In [ ]:
draft_5 = pd.DataFrame([w for w in flaubert_version[flaubert_version['version'] == '5']['token_text']])
draft_5 = draft_5.T
draft_5['Tokens'] = draft_5[0]
draft_5 = draft_5.drop(columns=[0])
draft_5.head()

In [ ]:
draft_6 = pd.DataFrame([w for w in flaubert_version[flaubert_version['version'] == '6']['token_text']])
draft_6 = draft_6.T
draft_6['Tokens'] = draft_6[0]
draft_6 = draft_6.drop(columns=[0])
draft_6.head()

In [ ]:
copiste = pd.DataFrame([w for w in flaubert_version[flaubert_version['version'] == 'C']['token_text']])
copiste = copiste.T
copiste['Tokens'] = copiste[0]
copiste = copiste.drop(columns=[0])
copiste.head()

In [ ]:
definitif = pd.DataFrame([w for w in flaubert_version[flaubert_version['version'] == 'D']['token_text']])
definitif = definitif.T
definitif['Tokens'] = definitif[0]
definitif = definitif.drop(columns=[0])
definitif.head()

In [ ]:
def len_punct(version):
    list_punct = list(string.punctuation)
    punct = version[version['Tokens'].isin(list_punct)]
    return len(punct)

In [ ]:
list_versions = [draft_1, draft_2, draft_3, draft_4, draft_5, draft_6, copiste, definitif]

In [ ]:
tot_punc_versions = [len_punct(version) for version in list_versions]

In [ ]:
tot_punc_versions

In [ ]:
plt.figure(figsize=[14, 10])
plt.title('Length of Total Punctuation in Each Version')
sns.barplot(x=versions, y=tot_punc_versions);

## Punctuation Compared: Margin and Unedited Punctuation

In [ ]:
len_punct_margin

In [ ]:
len_punct_previous_no_struck

In [ ]:
len_marg_unedit_punc = list(map(np.add, len_punct_margin, len_punct_previous_no_struck))

In [ ]:
len_marg_unedit_punc

In [ ]:
plt.figure(figsize=[14, 10])
plt.title('Quanitity Punctuation in Each Version')
sns.barplot(x=versions, y=len_marg_unedit_punc);

In [ ]:
draft_1_pos = st.tag(draft_1['Tokens'])
draft_1_pos[:10]

In [ ]:
from nltk.parse import CoreNLPParser

In [ ]:
pos_tagger = CoreNLPParser('http://localhost:9004', tagtype= 'pos')
pos_tagger.tag('Je suis enceinte'.split())
[('Je', 'PRON'), ('suis', 'VERB'), ('enceinte', 'ADJ')]

Feminine vs. Masculine writing -- La Martine as a feminine writer -- need to generate a bag of words to control for this -- Flaubert's earlier writings were more feminine